### Import Required Libraries and Set Up Environment Variables

In [309]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [310]:
# Set environment variables from the .env in the local environment
load_dotenv('environment.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

print(type(nyt_api_key))


<class 'str'>


### Access the New York Times API

In [311]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"


# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'
# filter_query = 'blockbuster'
# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
# field_list 
response_field = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
# https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey
# https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get

# query_url = f"{url}&fq={filter_query}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"

query_url = f"{url}" \
            f"&api-key={nyt_api_key}" \
            f"&fq={filter_query}" \
            f"&begin_date={begin_date}" \
            f"&end_date={end_date}" \
            f"&sort={sort}"\
            f"&fl={response_field}"

            # f"q={query}" \
            # f"&page={page}" \


In [312]:
# Make a Request and check response 
response = requests.get(query_url)
response

<Response [200]>

In [313]:

reviews_list = []           # list to store the reviews
review_count = 0;      
     
for page in range(0, 20):    # loop through pages 0-19
# create query with a page number; API results show 10 articles at a time
    query_url_by_page = f"{query_url}&page={str(page)}"

# Make a "GET" request and Retrieve Articles (retrieve the JSON)
    try:
# Request Reviews by page using URL Query above
        reviews = requests.get(query_url_by_page).json()    
# Loop through the response & save/append each article to the reviews_list
        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
            review_count += 1
# Print page that was just retrieved
        print(f'Successfully obtained {review_count} reviews, page {page}')
# If there is page number w/ no results, print page # and break from loop
    except:
        print(f'Failed to obtain reviews, page {page}')
        break

# Add 12 sec b/w queries to stay within API query limits
    time.sleep(12)

Successfully obtained 10 reviews, page 0
Successfully obtained 20 reviews, page 1
Successfully obtained 30 reviews, page 2
Successfully obtained 40 reviews, page 3
Successfully obtained 50 reviews, page 4
Successfully obtained 60 reviews, page 5
Successfully obtained 70 reviews, page 6
Successfully obtained 80 reviews, page 7
Successfully obtained 90 reviews, page 8
Successfully obtained 100 reviews, page 9
Successfully obtained 110 reviews, page 10
Successfully obtained 120 reviews, page 11
Successfully obtained 130 reviews, page 12
Successfully obtained 140 reviews, page 13
Successfully obtained 150 reviews, page 14
Successfully obtained 160 reviews, page 15
Successfully obtained 170 reviews, page 16
Successfully obtained 180 reviews, page 17
Successfully obtained 190 reviews, page 18
Successfully obtained 200 reviews, page 19


In [314]:
#  Print sample of reviews added to reviews_list
count = 0
print(f'Total reviews obtained: {len(reviews_list)}' )
for r in reviews_list:
    count +=1
    print(f"Sample review ({count}): {r['headline']['main']}") #snippet
    if count == 10:
        break

Total reviews obtained: 200
Sample review (1): ‘The Attachment Diaries’ Review: Love, Sick
Sample review (2): Review: ‘What’s Love Got to Do With It?’ Probably a Lot
Sample review (3): ‘You Can Live Forever’ Review: Do You Love Me Now?
Sample review (4): ‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Trip
Sample review (5): ‘Other People’s Children’ Review: True Romance
Sample review (6): ‘One True Loves’ Review: A Romance Lost at Sea
Sample review (7): ‘The Lost Weekend: A Love Story’ Review: When John Lennon Strayed
Sample review (8): ‘A Thousand and One’ Review: A New York Love Story
Sample review (9): ‘Your Place or Mine’ Review: Try Neither
Sample review (10): ‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouver


In [315]:
# Preview first 5 results in JSON format using json.dumps w/ argument indent=4 to format data
for i in range(5):
    print(f"\nJSON Sample {i} ({reviews_list[i]['headline']['main']})")
    print(json.dumps(reviews_list[i], indent=4))


JSON Sample 0 (‘The Attachment Diaries’ Review: Love, Sick)
{
    "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
    "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
    "source": "The New York Times",
    "headline": {
        "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "kicker": null,
        "content_kicker": null,
        "print_headline": "The Attachment Diaries",
        "name": null,
        "seo": null,
        "sub": null
    },
    "keywords": [
        {
            "name": "subject",
            "value": "Movies",
            "rank": 1,
            "major": "N"
        },
        {
            "name": "creative_works",
            "value": "The Attachment Diaries (Movie)",
            "rank": 2,
            "major": "N"
        },
        {
            "name": "persons",
            "value": "Diment, Valentin Javier",
      

In [316]:
# Use json_normalize to convert all_results to a DataFrame
reviews_df = pd.json_normalize(reviews_list)
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [317]:
# getting a list of columns
reviews_df.columns

Index(['web_url', 'snippet', 'source', 'keywords', 'pub_date', 'word_count',
       'headline.main', 'headline.kicker', 'headline.content_kicker',
       'headline.print_headline', 'headline.name', 'headline.seo',
       'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [318]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
#   Title is between unicode characters \u2018 and \u2019. 
#   End string should include " Review" to avoid cutting title early

# str : str[start, stop]
# str.find(sub[, start[, end]] )  # sub: substring to be searched in the string.

reviews_df['title'] = reviews_df['headline.main'].apply(\
        lambda str: str[str.find("\u2018")+1 : str.find("\u2019 Review")])
reviews_df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,Review: It’s All Mirth and Taxes in ‘Love & Ta...,None,None,"It’s Inevitable, Mirth and Taxes",None,None,None,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",None,None,Everybody Loves Somebody,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",Non

In [319]:

# Fix the "keywords" column by converting cells from a list to a string:
#   Use function `extract_keywords` function to convert the `"keywords"` column 
#   from a list of dictionaries to strings using the `apply()` method.

# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords


reviews_copy_df = reviews_df.copy()
reviews_copy_df['keywords'] = reviews_copy_df['keywords'].apply(extract_keywords)
reviews_copy_df.head()
        

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [320]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_copy_df['title'].to_list()
titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

In [321]:
print(f'total titles: {len(titles)}')
print(titles)

total titles: 200
['The Attachment Diaries', 'What’s Love Got to Do With It?’ Probably a Lo', 'You Can Live Forever', 'A Tourist’s Guide to Love', 'Other People’s Children', 'One True Loves', 'The Lost Weekend: A Love Story', 'A Thousand and One', 'Your Place or Mine', 'Love in the Time of Fentanyl', 'Pamela, a Love Story', 'In From the Side', 'After Love', 'Alcarràs', 'Nelly & Nadine', 'Lady Chatterley’s Lover', 'The Sound of Christmas', 'The Inspection', 'Bones and All', 'My Policeman', 'About Fate', 'Waiting for Bojangles', 'I Love My Dad', 'A Love Song', 'Alone Together', 'Art of Love', 'The Wheel', 'Thor: Love and Thunder', 'Both Sides of the Blade', 'Fire of Love', 'Love & Gelato', 'Stay Prayed Up', 'Benediction', 'Dinner in America', 'In a New York Minute', 'Anaïs in Love', 'I Love America', 'See You Then', 'La Mami', 'Love After Love', 'Deep Water', 'Lucy and Desi', 'Cyrano', 'The In Between', 'Book of Love', 'Lingui, the Sacred Bonds', 'The Pink Cloud', 'A Journal for Jordan',

### Access The Movie Database API

In [322]:
# Prepare The Movie Database query
# https://developer.themoviedb.org/docs/search-and-query-for-details

url = "https://api.themoviedb.org/3/search/movie?query="
print(type(tmdb_api_key))
tmdb_key_string = "&api_key=" + tmdb_api_key


<class 'str'>


In [323]:
# SAMPLE QUERY, JSON dump for formatting investigation.

# Search:
#  issue a query to one of the movie, TV show or person search methods
# 'https://api.themoviedb.org/3/search/movie?query=Jack+Reacher&api_key=API_KEY
#   look at is the 'results' field. 
#   This is an array and will contain standard movie list objects.
query = 'The Attachment Diaries'
query_url = f"{url}{query}{tmdb_key_string}"
print(query_url)
sample_movie_request = requests.get(query_url).json() 
# Use json.dumps with argument indent=4 to format data
# print(json.dumps(sample_movie_request, indent=4))


https://api.themoviedb.org/3/search/movie?query=The Attachment Diaries&api_key=a78f3a912736e51a4277ddf596af9b2f


In [324]:
# SAMPLE get movie id from JSON object

sample_movie_id = sample_movie_request["results"][0]["id"]
print(sample_movie_id)

# Make a request for a the full movie details using movie id
# url 'https://api.themoviedb.org/3/movie/343611?api_key=API_KEY'

# NEW QUERY URL !!
url_id = 'https://api.themoviedb.org/3/movie/'
id_query_url = f"{url_id}{sample_movie_id}?{tmdb_key_string}"
print(id_query_url)

# sample_movie_details_request = requests.get(id_query_url).json()
# print(json.dumps(sample_movie_details_request, indent=4))



743040
https://api.themoviedb.org/3/movie/743040?&api_key=a78f3a912736e51a4277ddf596af9b2f


In [400]:
# Create an empty list called tmdb_movies_list to store results from API requests. 
# This will contain a list of dictionaries aggregate of all the movies
#  so must be OUTSIDE of the loop.
tmdb_movies_list = []

# Create request counter to sleep requests after multiple of 50 requests
request_counter = 1

# Loop through the titles
for t in titles:
    # Check if we need to sleep before making a request
    # Use time.sleep(1) when it reaches a multiple of 50 and display a msg
    if request_counter % 50 == 0:
        print(f'zzz request counter: {request_counter}; commence sleep.')
        time.sleep(1)
    # Add 1 to the request counter
    request_counter += 1

    # # Limit to 5 for testing *********************************** 
    # if request_counter > 5:
    #     break

    # "GET" request for The Movie Database, query by movie, TV show or person:
    # 'https://api.themoviedb.org/3/search/movie?query=Jack+Reacher&api_key=API_KEY
    #   Include a try clause to search for the full movie details.
    #   Use the except clause to print out a statement if a movie is not found.
    #  These lists are inside the loop so they reset for each Movie.
    genre_list = []
    spoken_languages =[]
    production_countries = []   
    
    try:
        query_url = f"{url}{t}&{tmdb_key_string}"
        movie_request = requests.get(query_url).json()  
    
    # Get movie id
# Selecting values from JSON data requires data to be accessed first by the 
# parent object, and then child. When API returns output with multiple JSON 
# Objects, indexes have to be specified to # indicate which object/record should be selected
        movie_id = movie_request["results"][0]["id"]

    # Query For Full Movie Details using movie id:
    # url 'https://api.themoviedb.org/3/movie/343611?api_key=API_KEY'
    # NEW BASE QUERY URL !!
        id_url = 'https://api.themoviedb.org/3/movie/'

        id_query_url = f"{id_url}{movie_id}?{tmdb_key_string}"
        movie_details_request = requests.get(id_query_url).json()
        # See formatting of movie detail JSON
        # print(json.dumps(movie_details_request, indent=4))

#  Extract the genre names from the results into a list called genres.
        # append each genre to the genres_list
        for gen in (movie_details_request['genres']):
            genre_list.append(gen['name'])

#  Extract the spoken_languages' English name from results to list spoken_languages.
        for lang in (movie_details_request['spoken_languages']):
            spoken_languages.append(lang['name'])

#  Extract the production_countries' name from results to list production_countries.
        for country in (movie_details_request['production_countries']):
            production_countries.append(country['name'])
       
#  Create a dictionary with the following results: 
# `title`, `original_title`, `budget`, `original_language`, `homepage`, 
# `overview`, `popularity`, `runtime`, `revenue`, `release_date`, 
# `vote_average`, `vote_count`, 
#  as well as: `genres`, `spoken_languages`, `production_countries`
        movie_dictionary = {'title': movie_details_request['title'],
                            'original_title': movie_details_request['original_title'],  
                            'budget': movie_details_request['budget'], 
                              'genres': genre_list, 
                            'olanguage': movie_details_request['original_language'],
                              'spoken_languages': spoken_languages,  
                            'homepage': movie_details_request['homepage'], 
                            'overview': movie_details_request['overview'],  
                            'popularity': movie_details_request['popularity'], 
                            'runtime': movie_details_request['runtime'], 
                            'revenue': movie_details_request['revenue'], 
                            'release_date': movie_details_request['release_date'], 
                            'vote_average': movie_details_request['vote_average'], 
                            'vote_count': movie_details_request['vote_count'], 
                              'production_countries': production_countries}
                            
#  Append dictionary to `tmdb_movies_list`
        tmdb_movies_list.append(movie_dictionary)

#  Print out the name of the movie and a message to indicate that the title was found.
        print(f'Found: {t}; movie id: {movie_id}')

    except:
        print(f"{t} ('movie id: {movie_id}): NOT FOUND ")


Found: The Attachment Diaries; movie id: 743040
What’s Love Got to Do With It?’ Probably a Lo ('movie id: 743040): NOT FOUND 
Found: You Can Live Forever; movie id: 887580
Found: A Tourist’s Guide to Love; movie id: 813726
Found: Other People’s Children; movie id: 803694
Found: One True Loves; movie id: 846961
Found: The Lost Weekend: A Love Story; movie id: 965042
Found: A Thousand and One; movie id: 855263
Found: Your Place or Mine; movie id: 703451
Found: Love in the Time of Fentanyl; movie id: 976653
Found: Pamela, a Love Story; movie id: 1061671
Found: In From the Side; movie id: 622082
Found: After Love; movie id: 714011
Found: Alcarràs; movie id: 804251
Found: Nelly & Nadine; movie id: 12652
Found: Lady Chatterley’s Lover; movie id: 814338
Found: The Sound of Christmas; movie id: 1035923
Found: The Inspection; movie id: 837938
Found: Bones and All; movie id: 791177
Found: My Policeman; movie id: 744114
Found: About Fate; movie id: 828613
Found: Waiting for Bojangles; movie id: 8

In [401]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data

for i in range(5):
    print(json.dumps(tmdb_movies_list[i], indent=4))

{
    "title": "The Attachment Diaries",
    "original_title": "El apego",
    "budget": 0,
    "genres": [
        "Drama",
        "Mystery",
        "Thriller",
        "Horror"
    ],
    "olanguage": "es",
    "spoken_languages": [
        "Espa\u00f1ol"
    ],
    "homepage": "",
    "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
    "popularity": 2.89,
    "runtime": 102,
    "revenue": 0,
    "release_date": "2021-10-07",
    "vote_average": 3.0,
    "vote_count": 4,
    "production_countries": [
        "Argentina"
    ]
}
{
    "title": "You Can Live Forever",
    "original_title": "You Can Live Forever",
    "budget": 0,
   

In [402]:
# Convert the results to a Pandas DataFrame using json_normalize()
moviedb_df = pd.json_normalize(tmdb_movies_list)
moviedb_df.head(10)

,title,original_title,budget,genres,olanguage,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Español],,"Argentina, 1970s. A desperate young woman goes...",2.890,102,0,2021-10-07,3.000,4,[Argentina]
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,15055,2023-03-24,6.622,37,"[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,"[Romance, Comedy]",en,"[English, Tiếng Việt]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",12.581,96,0,2023-04-21,6.289,161,[United States of America]
3,Other People's Children,Les Enfants des autres,0,"[Drama, Comedy]",fr,"[Français, English]",https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",10.518,104,84178,2022-09-21,6.818,190,[France]
4,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Español]",,Emma and Jesse are living the perfect life tog...,11.056,100,37820,2023-04-07,6.500,74,"[Czech Republic, United States of America]"
5,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.466,95,77145,2023-04-13,6.000,2,[United States of America]
6,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Español, Português]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,8.610,116,3395595,2023-03-31,6.871,89,[United States of America]
7,Your Place or Mine,Your Place or Mine,0,"[Romance, Comedy]",en,[English],https://www.netflix.com/title/81045831,When best friends and total opposites Debbie a...,16.511,109,0,2023-02-10,6.280,710,[United States of America]
8,Love in the Time of Fentanyl,Love in the Time of Fentanyl,0,[Documentary],en,[English],,An intimate portrait of a community fighting t...,0.924,85,0,2023-02-03,0.000,0,"[Canada, United States of America]"
9,"Pamela, A Love Story","Pamela, A Love Story",0,[Documentary],en,[English],https://www.netflix.com/title/81590934,"In her own words, through personal video and d...",9.427,113,0,2023-01-30,6.978,207,[United States of America]


### Merge and Clean the Data for Export

In [408]:
# Merge the New York Times reviews and TMDB DataFrames on title
movies_combined_df = moviedb_df.merge(reviews_copy_df, on='title')
movies_combined_df


,title,original_title,budget,genres,olanguage,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",es,[Español],,"Argentina, 1970s. A desperate young woman goes...",2.890,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",en,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",36.659,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"[Romance, Drama, Comedy]",en,"[English, Español]",,Emma and Jesse are living the perfect life tog...,11.056,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,[Documentary],en,[English],https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,2.466,95,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,"[Drama, Crime]",en,"[English, Español, Português]",https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,8.610,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,The Last Face,The Last Face,0,[Drama],en,"[English, Português]",,"Miguel, a heroic Spanish doctor, puts himself ...",7.490,130,...,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
123,Lost in Paris,Paris pieds nus,0,[Comedy],fr,[Français],,Fiona visits Paris for the first time to assis...,6.823,83,...,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,"[{'firstname': 'Ben', 'middlename': None, 'las...",None
124,The Other Half,The Other Half,0,"[Drama, Romance]",en,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,2.871,103,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
125,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",en,"[Türkçe, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",16.901,111,...,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None


In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [ ]:
# Drop "byline.person" column


In [ ]:
# Delete duplicate rows and reset index


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [ ]:
# Export data to CSV without the index
